In [1]:
import argparse
import os
import json
import sys
from codebase.dataset import Dataset
from codebase.models import *
from codebase.trainer import Trainer, make_dir_if_not_exist
from codebase.defaults import get_default_kwargs
import tensorflow as tf


2024-01-15 16:59:59.671249: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 16:59:59.727894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 16:59:59.727958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 16:59:59.729561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 16:59:59.740109: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 16:59:59.741093: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
parser = argparse.ArgumentParser(description='Run model')
parser.add_argument('-d', '--dataset', help='dataset name', default='bank_age')
parser.add_argument('-n', '--expname', help='experiment name', default='temp')
parser.add_argument('-pc', '--pass_coeff', help='coefficient on pass term', default=1., type=float)
parser.add_argument('-fc', '--fair_coeff', help='coefficient on fairness term', default=1., type=float)
parser.add_argument('-pass', '--pass_model', action='store_true', help='run PASS model?', default=False)
parser.add_argument('-def', '--defer', action='store_true', help='learn to defer? (as opposed to reject)', default=True)
parser.add_argument('-ne', '--num_epochs', help='number of training epochs', default=10000, type=int)
parser.add_argument('-pa', '--patience', help='training patience for early stopping', default=30, type=int)
parser.add_argument('-ua', '--use_attr', action='store_true', help='use sensitive attribute in data', default=True)
parser.add_argument('-mrs', '--model_random_seed', help='what random seed to use for model', default=0, type=int)
parser.add_argument('-drs', '--data_random_seed', help='what random seed to use for data', default=0, type=int)
parser.add_argument('-dm', '--dm_type', help='type of DM to use: inconsistent/biased/highacc', default='inconsistent')
parser.add_argument('-dirs', '--dirconf', help='config file for dirs', default='conf/dirs/home.json')
args = vars(parser.parse_args([]))


In [3]:
#load options
dirs = json.load(open(args['dirconf'], 'r'))
model_args = json.load(open('conf/model/{}.json'.format(args['dataset'])))

# get default params
data_kwargs, model_kwargs = get_default_kwargs(dirs['data'], args, model_args)

In [6]:
#change data kwargs
experiment_info = {}
experiment_info["K"] = []
experiment_info["budget"] = []
experiment_info["Error"] = []
experiment_info["Equal opportunity difference"] = []
experiment_info["fc"] = []
#fcs = [1.5,2,3,4,5]
fcs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]
for fc in fcs:
    model_kwargs["fair_coeff"] = fc
    for K in [1,2,3,4,5]:
        for budget in [1, 0.1, 0.01]:
            tf.compat.v1.reset_default_graph()
            data_kwargs['npzfile'] = "data/bank/bank{}_age.npz".format(K,budget)
            #get dataset
            data = Dataset(**data_kwargs,budget=budget)
            model = MLPDeferModel(**model_kwargs)

            with tf.compat.v1.Session() as sess:
                print('Session created.')
                resdirname = os.path.join(dirs['exp'], args['dataset'])
                logdirname = os.path.join(dirs['log'], args['dataset'])
                ckptdirname = os.path.join(resdirname, 'checkpoints')
                for d in [resdirname, logdirname, ckptdirname]:
                    make_dir_if_not_exist(d)
                #create Trainer
                trainer = Trainer(model, data, sess=sess, logs_path=logdirname, \
                            checkpoint_path=ckptdirname, results_path=resdirname)

                save_path = trainer.train(n_epochs=args['num_epochs'], patience=args['patience'])
                trainer.restore(save_path)
                error,eod = trainer.test()
                print('Complete.')
                experiment_info["K"].append(K)
                experiment_info["budget"].append(budget)
                experiment_info["Error"].append(error)
                experiment_info["Equal opportunity difference"].append(eod)
                experiment_info["fc"].append(fc)


Session created.
Train: E0: loss:0.334, class_loss:0.254, fair_reg:0.482, idk_loss:0.080, loss_class:0.254, loss_idk:0.056, errRate:0.107, DI:0.068, errRate-T:0.106, DI-T:0.072, rejErrRate:0.106, IDKRate:0.041 | Valid: E0: loss:0.235, class_loss:0.229, fair_reg:0.453, idk_loss:0.006, loss_class:0.229, loss_idk:0.005, errRate:0.105, DI:0.152, errRate-T:0.105, DI-T:0.152, rejErrRate:0.105, IDKRate:0.000
Best validation loss so far of 0.235, model saved to /home/hc838/AuFair/predict-responsibly/experiments/bank_age/checkpoints/model.ckpt
Train: E1: loss:0.231, class_loss:0.227, fair_reg:0.453, idk_loss:0.004, loss_class:0.227, loss_idk:0.003, errRate:0.103, DI:0.079, errRate-T:0.103, DI-T:0.079, rejErrRate:0.103, IDKRate:0.000 | Valid: E1: loss:0.227, class_loss:0.225, fair_reg:0.458, idk_loss:0.002, loss_class:0.225, loss_idk:0.002, errRate:0.104, DI:0.169, errRate-T:0.104, DI-T:0.169, rejErrRate:0.104, IDKRate:0.000
Train: E2: loss:0.224, class_loss:0.223, fair_reg:0.448, idk_loss:0.002

Train: E21: loss:0.204, class_loss:0.204, fair_reg:0.422, idk_loss:0.000, loss_class:0.204, loss_idk:0.000, errRate:0.096, DI:0.096, errRate-T:0.096, DI-T:0.096, rejErrRate:0.096, IDKRate:0.000 | Valid: E21: loss:0.235, class_loss:0.235, fair_reg:0.524, idk_loss:0.000, loss_class:0.235, loss_idk:0.000, errRate:0.103, DI:0.090, errRate-T:0.103, DI-T:0.090, rejErrRate:0.103, IDKRate:0.000
Train: E22: loss:0.203, class_loss:0.203, fair_reg:0.421, idk_loss:0.000, loss_class:0.203, loss_idk:0.000, errRate:0.096, DI:0.095, errRate-T:0.096, DI-T:0.095, rejErrRate:0.096, IDKRate:0.000 | Valid: E22: loss:0.236, class_loss:0.236, fair_reg:0.525, idk_loss:0.000, loss_class:0.236, loss_idk:0.000, errRate:0.102, DI:0.103, errRate-T:0.102, DI-T:0.103, rejErrRate:0.102, IDKRate:0.000
Train: E23: loss:0.203, class_loss:0.203, fair_reg:0.421, idk_loss:0.000, loss_class:0.203, loss_idk:0.000, errRate:0.096, DI:0.102, errRate-T:0.096, DI-T:0.102, rejErrRate:0.096, IDKRate:0.000 | Valid: E23: loss:0.236, 

Train: E3: loss:0.283, class_loss:0.254, fair_reg:0.443, idk_loss:0.029, loss_class:0.254, loss_idk:0.021, errRate:0.103, DI:0.120, errRate-T:0.103, DI-T:0.120, rejErrRate:0.103, IDKRate:0.000 | Valid: E3: loss:0.287, class_loss:0.265, fair_reg:0.569, idk_loss:0.023, loss_class:0.265, loss_idk:0.017, errRate:0.085, DI:0.313, errRate-T:0.085, DI-T:0.313, rejErrRate:0.085, IDKRate:0.000
Train: E4: loss:0.264, class_loss:0.244, fair_reg:0.421, idk_loss:0.019, loss_class:0.244, loss_idk:0.015, errRate:0.101, DI:0.103, errRate-T:0.101, DI-T:0.103, rejErrRate:0.101, IDKRate:0.000 | Valid: E4: loss:0.280, class_loss:0.263, fair_reg:0.572, idk_loss:0.016, loss_class:0.263, loss_idk:0.012, errRate:0.092, DI:0.327, errRate-T:0.092, DI-T:0.327, rejErrRate:0.092, IDKRate:0.000
Train: E5: loss:0.254, class_loss:0.239, fair_reg:0.410, idk_loss:0.014, loss_class:0.239, loss_idk:0.011, errRate:0.100, DI:0.097, errRate-T:0.100, DI-T:0.097, rejErrRate:0.100, IDKRate:0.000 | Valid: E5: loss:0.279, class_

Train: E23: loss:0.209, class_loss:0.207, fair_reg:0.370, idk_loss:0.002, loss_class:0.207, loss_idk:0.001, errRate:0.088, DI:0.110, errRate-T:0.088, DI-T:0.110, rejErrRate:0.088, IDKRate:0.000 | Valid: E23: loss:0.267, class_loss:0.265, fair_reg:0.586, idk_loss:0.002, loss_class:0.265, loss_idk:0.001, errRate:0.087, DI:0.324, errRate-T:0.087, DI-T:0.324, rejErrRate:0.087, IDKRate:0.000
Train: E24: loss:0.207, class_loss:0.206, fair_reg:0.367, idk_loss:0.001, loss_class:0.206, loss_idk:0.001, errRate:0.087, DI:0.147, errRate-T:0.087, DI-T:0.147, rejErrRate:0.087, IDKRate:0.000 | Valid: E24: loss:0.267, class_loss:0.265, fair_reg:0.585, idk_loss:0.001, loss_class:0.265, loss_idk:0.001, errRate:0.087, DI:0.324, errRate-T:0.087, DI-T:0.324, rejErrRate:0.087, IDKRate:0.000
Best validation loss so far of 0.267, model saved to /home/hc838/AuFair/predict-responsibly/experiments/bank_age/checkpoints/model.ckpt
Train: E25: loss:0.205, class_loss:0.204, fair_reg:0.366, idk_loss:0.001, loss_class

Train: E44: loss:0.177, class_loss:0.176, fair_reg:0.339, idk_loss:0.001, loss_class:0.176, loss_idk:0.000, errRate:0.078, DI:0.140, errRate-T:0.078, DI-T:0.140, rejErrRate:0.078, IDKRate:0.000 | Valid: E44: loss:0.275, class_loss:0.275, fair_reg:0.580, idk_loss:0.001, loss_class:0.275, loss_idk:0.000, errRate:0.090, DI:0.316, errRate-T:0.090, DI-T:0.316, rejErrRate:0.090, IDKRate:0.000
Train: E45: loss:0.175, class_loss:0.175, fair_reg:0.337, idk_loss:0.001, loss_class:0.175, loss_idk:0.000, errRate:0.078, DI:0.138, errRate-T:0.078, DI-T:0.138, rejErrRate:0.078, IDKRate:0.000 | Valid: E45: loss:0.277, class_loss:0.276, fair_reg:0.580, idk_loss:0.001, loss_class:0.276, loss_idk:0.000, errRate:0.087, DI:0.304, errRate-T:0.087, DI-T:0.304, rejErrRate:0.087, IDKRate:0.000
Train: E46: loss:0.174, class_loss:0.173, fair_reg:0.335, idk_loss:0.001, loss_class:0.173, loss_idk:0.000, errRate:0.077, DI:0.138, errRate-T:0.077, DI-T:0.138, rejErrRate:0.077, IDKRate:0.000 | Valid: E46: loss:0.278, 

...

Train: E34: loss:0.439, class_loss:0.274, fair_reg:0.164, idk_loss:0.001, loss_class:0.274, loss_idk:0.000, errRate:0.131, DI:0.084, errRate-T:0.131, DI-T:0.084, rejErrRate:0.131, IDKRate:0.000 | Valid: E34: loss:0.783, class_loss:0.402, fair_reg:0.380, idk_loss:0.001, loss_class:0.402, loss_idk:0.000, errRate:0.172, DI:0.148, errRate-T:0.172, DI-T:0.148, rejErrRate:0.172, IDKRate:0.000
Train: E35: loss:0.438, class_loss:0.270, fair_reg:0.167, idk_loss:0.001, loss_class:0.270, loss_idk:0.000, errRate:0.129, DI:0.081, errRate-T:0.129, DI-T:0.081, rejErrRate:0.129, IDKRate:0.000 | Valid: E35: loss:0.799, class_loss:0.414, fair_reg:0.384, idk_loss:0.001, loss_class:0.414, loss_idk:0.000, errRate:0.169, DI:0.138, errRate-T:0.169, DI-T:0.138, rejErrRate:0.169, IDKRate:0.000
Train: E36: loss:0.439, class_loss:0.275, fair_reg:0.164, idk_loss:0.001, loss_class:0.275, loss_idk:0.000, errRate:0.133, DI:0.057, errRate-T:0.133, DI-T:0.057, rejErrRate:0.133, IDKRate:0.000 | Valid: E36: loss:0.787, 

Train: E17: loss:0.606, class_loss:0.284, fair_reg:0.249, idk_loss:0.072, loss_class:0.284, loss_idk:0.052, errRate:0.141, DI:0.467, errRate-T:0.141, DI-T:0.467, rejErrRate:0.141, IDKRate:0.000 | Valid: E17: loss:1.172, class_loss:0.495, fair_reg:0.610, idk_loss:0.067, loss_class:0.495, loss_idk:0.034, errRate:0.256, DI:nan, errRate-T:0.256, DI-T:nan, rejErrRate:0.256, IDKRate:0.000
Train: E18: loss:0.590, class_loss:0.282, fair_reg:0.244, idk_loss:0.064, loss_class:0.282, loss_idk:0.046, errRate:0.141, DI:0.467, errRate-T:0.141, DI-T:0.467, rejErrRate:0.141, IDKRate:0.000 | Valid: E18: loss:1.131, class_loss:0.506, fair_reg:0.564, idk_loss:0.060, loss_class:0.506, loss_idk:0.030, errRate:0.256, DI:nan, errRate-T:0.256, DI-T:nan, rejErrRate:0.256, IDKRate:0.000
Best validation loss so far of 1.131, model saved to /home/hc838/AuFair/predict-responsibly/experiments/bank_age/checkpoints/model.ckpt
Train: E19: loss:0.560, class_loss:0.283, fair_reg:0.219, idk_loss:0.058, loss_class:0.283, 

Train: E38: loss:0.342, class_loss:0.207, fair_reg:0.119, idk_loss:0.016, loss_class:0.207, loss_idk:0.013, errRate:0.109, DI:0.113, errRate-T:0.109, DI-T:0.113, rejErrRate:0.109, IDKRate:0.000 | Valid: E38: loss:1.049, class_loss:0.541, fair_reg:0.493, idk_loss:0.015, loss_class:0.541, loss_idk:0.007, errRate:0.256, DI:nan, errRate-T:0.256, DI-T:nan, rejErrRate:0.256, IDKRate:0.000
Train: E39: loss:0.311, class_loss:0.205, fair_reg:0.091, idk_loss:0.015, loss_class:0.205, loss_idk:0.012, errRate:0.109, DI:0.113, errRate-T:0.109, DI-T:0.113, rejErrRate:0.109, IDKRate:0.000 | Valid: E39: loss:1.048, class_loss:0.542, fair_reg:0.492, idk_loss:0.015, loss_class:0.542, loss_idk:0.007, errRate:0.256, DI:nan, errRate-T:0.256, DI-T:nan, rejErrRate:0.256, IDKRate:0.000
Train: E40: loss:0.342, class_loss:0.199, fair_reg:0.129, idk_loss:0.014, loss_class:0.199, loss_idk:0.011, errRate:0.103, DI:0.109, errRate-T:0.103, DI-T:0.109, rejErrRate:0.103, IDKRate:0.000 | Valid: E40: loss:1.101, class_lo

Train: E59: loss:0.216, class_loss:0.136, fair_reg:0.073, idk_loss:0.007, loss_class:0.136, loss_idk:0.006, errRate:0.071, DI:0.104, errRate-T:0.071, DI-T:0.104, rejErrRate:0.071, IDKRate:0.000 | Valid: E59: loss:1.051, class_loss:0.543, fair_reg:0.501, idk_loss:0.007, loss_class:0.543, loss_idk:0.003, errRate:0.231, DI:nan, errRate-T:0.231, DI-T:nan, rejErrRate:0.231, IDKRate:0.000
Train: E60: loss:0.203, class_loss:0.134, fair_reg:0.063, idk_loss:0.007, loss_class:0.134, loss_idk:0.006, errRate:0.064, DI:0.101, errRate-T:0.064, DI-T:0.101, rejErrRate:0.064, IDKRate:0.000 | Valid: E60: loss:1.051, class_loss:0.543, fair_reg:0.501, idk_loss:0.007, loss_class:0.543, loss_idk:0.003, errRate:0.231, DI:nan, errRate-T:0.231, DI-T:nan, rejErrRate:0.231, IDKRate:0.000
Train: E61: loss:0.218, class_loss:0.133, fair_reg:0.079, idk_loss:0.007, loss_class:0.133, loss_idk:0.006, errRate:0.064, DI:0.101, errRate-T:0.064, DI-T:0.101, rejErrRate:0.064, IDKRate:0.000 | Valid: E61: loss:1.056, class_lo

0.6999999998249999
0.6586741889892994
Error rate is 0.16237493849434148
Equal opportunity difference is 0.04132581083570053
Test: Test: loss:0.731, class_loss:0.369, fair_reg:0.352, idk_loss:0.009, loss_class:0.369, loss_idk:0.006, errRate:0.162, DI:0.033, errRate-T:0.162, DI-T:0.033, rejErrRate:0.162, IDKRate:0.000
Metrics saved to /home/hc838/AuFair/predict-responsibly/experiments/bank_age/test_results.csv
Tensors saved to /home/hc838/AuFair/predict-responsibly/experiments/bank_age
Complete.


In [8]:
import pandas as pd
df = pd.DataFrame.from_dict(experiment_info)
#save df to csv
df.to_csv("results/bank_age.csv",index=False)